In [1]:
import numpy as np
from lookup_table import CaseNum2EdgeOffset, getCaseNum
import trimesh
import os
import time

In [2]:
def marching_cube(thres,cells):
    '''
    cells: (64, 64, 64)
    '''
    # vertices use dictionary to avoid duplicate axes
    vertex_array = {}
    face_array = []
    t1 = time.time()
    # -------------------TODO------------------ 
    # compute vertices and faces
    # vertices: [N, 3]
    # faces: [M, 3], e.g. np.array([[0,1,2]]) means a triangle composed of vertices[0], vertices[1] and vertices[2]
    # for-loop is allowed to reduce difficulty
    # -------------------TODO------------------ 
    # 为了使索引值对应，建立edge->index->vertex的映射
    edge2ind = {}
    H,W,D = cells.shape
    cnt = 0
    tmp = [] # Use every three consecutive interaction points to form a face.
    for x in range(H-1):
        for y in range(W-1):
            for z in range(D-1):
                # 1. Use "getCaseNum(x,y,z,thre,cells)" to get a list of case values for this cell.
                case_nums = getCaseNum(x,y,z,thres,cells)
                for case_num in case_nums:
                    # 2. For each case value "case_num", use "CaseNum2EdgeOffset[case_num]" 
                    # to get an edge of this cell which interacts with the iso-surface.
                    if case_num < 0:
                        break
                    offset = CaseNum2EdgeOffset[case_num]
                    corner1 = (x+offset[0],y+offset[1],z+offset[2])
                    corner2 = (x+offset[3],y+offset[4],z+offset[5])
                    if corner1[0]>corner2[0] or corner1[1]>corner2[1] or corner1[2]>corner2[2]:
                        corner1,corner2 = corner2,corner1
                    # 3. Use linear interpolation to find the interaction point "(xp,yp,zp)"
                    if (corner1,corner2) not in edge2ind.keys():
                        edge2ind[(corner1,corner2)] = cnt
                        v1 = np.abs(cells[corner1])
                        v2 = np.abs(cells[corner2])
                        vertex = np.array(corner1)*(v2/(v1+v2)) + np.array(corner2)*(v1/(v1+v2))
                        vertex_array[cnt] = vertex
                        cnt += 1
                    tmp.append(edge2ind[(corner1,corner2)])
                    if len(tmp)==3:
                        face_array.append(tmp)
                        tmp = []
    
    t2 = time.time()
    print("\nTime taken by algorithm\n"+'-'*40+"\n{} s".format(t2-t1))
    #vertex_array = list(vertex_array.values())
    vertex_array = [vertex_array[i] for i in range(cnt)]
    return np.array(vertex_array), np.array(face_array)

In [3]:
# reconstruct these two animals
shape_name_lst = ['spot', 'bob']
for shape_name in shape_name_lst:
    data = np.load(os.path.join('data', shape_name + '_cell.npy'))
    verts, faces = marching_cube(0, data)
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    mesh_txt = trimesh.exchange.obj.export_obj(mesh)
    with open(os.path.join('../results', shape_name + '.obj'),"w") as fp:
        fp.write(mesh_txt)


Time taken by algorithm
----------------------------------------
7.516629934310913 s

Time taken by algorithm
----------------------------------------
7.796140432357788 s
